In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load cleaned merged data
df = pd.read_csv("/Users/erion/Desktop/MSCF2/ADA/Capstone-Project/data/processed/merged_dataset.csv", parse_dates=['Date'], index_col='Date')

df.head()

,gold,eurusd,treasury_10y,spy,vix,dxy,oil
Date,,,,,,,
2003-12-03,0.000495,0.002813,4.410,-0.001584,0.022127,-0.001562,0.010396
2003-12-04,-0.001486,-0.003467,4.369,0.004106,-0.019844,0.001117,0.005145
2003-12-05,0.007687,0.008775,4.215,-0.006970,0.048466,-0.005134,-0.016955
2003-12-08,0.000492,0.002713,4.278,0.006738,-0.032183,-0.004375,0.044582
2003-12-09,0.003935,0.002450,4.352,-0.007716,0.065901,-0.001577,-0.010592


In [28]:
# Target: next-day gold return
df["target"] = df["gold"].shift(-1)

df.tail()

,gold,eurusd,treasury_10y,spy,vix,dxy,oil,target
Date,,,,,,,,
2024-12-24,0.002948,-0.002612,4.591,0.011115,-0.149583,0.002036,0.012421,0.000000
2024-12-25,0.000000,-0.000312,4.591,0.000000,0.000000,0.000000,0.000000,0.007176
2024-12-26,0.007176,-0.000291,4.579,0.000067,0.032235,-0.001201,-0.006847,-0.008186
2024-12-27,-0.008186,0.002272,4.619,-0.010527,0.082824,-0.001202,0.014076,-0.004241
2024-12-30,-0.004241,0.000594,4.545,-0.011412,0.090909,0.001204,0.005524,NaN


In [29]:
# Lagged gold returns
for lag in range(1, 6):
    df[f"gold_lag{lag}"] = df["gold"].shift(lag)

print(df[[f"gold_lag{i}" for i in range(1, 6)]].head())

            gold_lag1  gold_lag2  gold_lag3  gold_lag4  gold_lag5
Date                                                             
2003-12-03        NaN        NaN        NaN        NaN        NaN
2003-12-04   0.000495        NaN        NaN        NaN        NaN
2003-12-05  -0.001486   0.000495        NaN        NaN        NaN
2003-12-08   0.007687  -0.001486   0.000495        NaN        NaN
2003-12-09   0.000492   0.007687  -0.001486   0.000495        NaN


In [30]:
# Rolling volatility features (10, 20, 30 days)
windows = [10, 20, 30]

for w in windows:
    df[f"vol_{w}"] = df["gold"].rolling(w).std()

In [31]:
for w in windows:
    df[f"ma_{w}"] = df["gold"].rolling(w).mean()

In [32]:
for w in windows:
    df[f"momentum_{w}"] = df["gold"].rolling(w).sum()

In [33]:
def compute_RSI(series, window=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

df["rsi_14"] = compute_RSI(df["gold"])
df["rsi_14"].head()

Date
2003-12-03   NaN
2003-12-04   NaN
2003-12-05   NaN
2003-12-08   NaN
2003-12-09   NaN
Name: rsi_14, dtype: float64

In [34]:
ema12 = df["gold"].ewm(span=12, adjust=False).mean()
ema26 = df["gold"].ewm(span=26, adjust=False).mean()

df["macd"] = ema12 - ema26
df["macd_signal"] = df["macd"].ewm(span=9, adjust=False).mean()

In [35]:
# Summary of all features created
print("=" * 70)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 70)
print(f"\nTotal rows: {len(df)}")
print(f"Date range: {df.index[0].date()} to {df.index[-1].date()}")
print(f"\nTotal columns: {df.shape[1]}")
print(f"Total features (excluding target): {df.shape[1] - 1}")

print("\n--- FEATURE BREAKDOWN ---")
print(f"1. Target variable: 1 (next-day gold return)")
print(f"2. Lagged returns: 5 (gold_lag1 to gold_lag5)")
print(f"3. Volatility features: 3 (vol_10, vol_20, vol_30)")
print(f"4. Moving averages: 3 (ma_10, ma_20, ma_30)")
print(f"5. Momentum features: 3 (momentum_10, momentum_20, momentum_30)")
print(f"6. RSI: 1 (rsi_14)")
print(f"7. MACD: 2 (macd, macd_signal)")
print(f"\nTotal features: {5 + 3 + 3 + 3 + 1 + 2} = 17 features + 1 target")

print("\n--- DATA QUALITY ---")
print(f"Missing values per column:")
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0].sort_values(ascending=False))
else:
    print("No missing values!")

print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows with all features:")
print(df.head(20))


FEATURE ENGINEERING SUMMARY

Total rows: 5499
Date range: 2003-12-03 to 2024-12-30

Total columns: 25
Total features (excluding target): 24

--- FEATURE BREAKDOWN ---
1. Target variable: 1 (next-day gold return)
2. Lagged returns: 5 (gold_lag1 to gold_lag5)
3. Volatility features: 3 (vol_10, vol_20, vol_30)
4. Moving averages: 3 (ma_10, ma_20, ma_30)
5. Momentum features: 3 (momentum_10, momentum_20, momentum_30)
6. RSI: 1 (rsi_14)
7. MACD: 2 (macd, macd_signal)

Total features: 17 = 17 features + 1 target

--- DATA QUALITY ---
Missing values per column:
vol_30         29
ma_30          29
momentum_30    29
vol_20         19
ma_20          19
momentum_20    19
rsi_14         14
vol_10          9
ma_10           9
momentum_10     9
gold_lag5       5
gold_lag4       4
gold_lag3       3
gold_lag2       2
target          1
gold_lag1       1
dtype: int64

DataFrame shape: (5499, 25)

First few rows with all features:
                gold    eurusd  treasury_10y       spy       vix       dxy

In [36]:
# All column names (features + target)
print("ALL FEATURES IN YOUR DATASET:")
print("=" * 50)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nTotal: {len(df.columns)} columns")
print(f"Target variable: 'target'")
print(f"Features (excluding target): {len(df.columns) - 1}")


ALL FEATURES IN YOUR DATASET:
 1. gold
 2. eurusd
 3. treasury_10y
 4. spy
 5. vix
 6. dxy
 7. oil
 8. target
 9. gold_lag1
10. gold_lag2
11. gold_lag3
12. gold_lag4
13. gold_lag5
14. vol_10
15. vol_20
16. vol_30
17. ma_10
18. ma_20
19. ma_30
20. momentum_10
21. momentum_20
22. momentum_30
23. rsi_14
24. macd
25. macd_signal

Total: 25 columns
Target variable: 'target'
Features (excluding target): 24


In [37]:
# Exogenous lagged features (DXY, VIX, SPY, Oil, Treasury, EUR/USD)
exogenous_assets = ['eurusd', 'spy', 'vix', 'dxy', 'oil', 'treasury_10y']

for asset in exogenous_assets:
    for lag in range(1, 4):
        df[f"{asset}_lag{lag}"] = df[asset].shift(lag)

print("Exogenous lagged features created for all assets")
print(f"Total columns now: {df.shape[1]}")


Exogenous lagged features created for all assets
Total columns now: 43


In [38]:
# Final feature engineering summary
print("=" * 70)
print("FINAL FEATURE ENGINEERING SUMMARY")
print("=" * 70)

print(f"\n✓ Total rows: {len(df)}")
print(f"✓ Date range: {df.index[0].date()} to {df.index[-1].date()}")
print(f"✓ Total columns: {df.shape[1]}")
print(f"✓ Total features (excluding target): {df.shape[1] - 1}")

print("\n--- FEATURE BREAKDOWN ---")
print("Endogenous (Gold) Features:")
print(f"  - Lagged returns: 5 (gold_lag1-5)")
print(f"  - Volatility: 3 (vol_10, vol_20, vol_30)")
print(f"  - Moving averages: 3 (ma_10, ma_20, ma_30)")
print(f"  - Momentum: 3 (momentum_10, momentum_20, momentum_30)")
print(f"  - RSI: 1 (rsi_14)")
print(f"  - MACD: 2 (macd, macd_signal)")
print(f"  Subtotal: 17 features")

print("\nExogenous Features (Lagged):")
print(f"  - EUR/USD: 3 lags")
print(f"  - S&P 500: 3 lags")
print(f"  - VIX: 3 lags")
print(f"  - DXY: 3 lags")
print(f"  - WTI Oil: 3 lags")
print(f"  - 10Y Treasury: 3 lags")
print(f"  Subtotal: 18 features")

print("\nTarget:")
print(f"  - Next-day gold return: 1")

print(f"\nTOTAL FEATURES: 17 + 18 + 1 = {df.shape[1]}")

print("\n--- DATA QUALITY ---")
missing = df.isnull().sum()
print(f"Missing values: {missing.sum()} total")
print(f"Rows available for modeling (after dropna): {len(df.dropna())}")
print(f"Data retention: {(len(df.dropna())/len(df))*100:.1f}%")

print(f"\nDataFrame shape: {df.shape}")
print("\nReady for walk-forward validation and modeling! ✓")


FINAL FEATURE ENGINEERING SUMMARY

✓ Total rows: 5499
✓ Date range: 2003-12-03 to 2024-12-30
✓ Total columns: 43
✓ Total features (excluding target): 42

--- FEATURE BREAKDOWN ---
Endogenous (Gold) Features:
  - Lagged returns: 5 (gold_lag1-5)
  - Volatility: 3 (vol_10, vol_20, vol_30)
  - Moving averages: 3 (ma_10, ma_20, ma_30)
  - Momentum: 3 (momentum_10, momentum_20, momentum_30)
  - RSI: 1 (rsi_14)
  - MACD: 2 (macd, macd_signal)
  Subtotal: 17 features

Exogenous Features (Lagged):
  - EUR/USD: 3 lags
  - S&P 500: 3 lags
  - VIX: 3 lags
  - DXY: 3 lags
  - WTI Oil: 3 lags
  - 10Y Treasury: 3 lags
  Subtotal: 18 features

Target:
  - Next-day gold return: 1

TOTAL FEATURES: 17 + 18 + 1 = 43

--- DATA QUALITY ---
Missing values: 237 total
Rows available for modeling (after dropna): 5469
Data retention: 99.5%

DataFrame shape: (5499, 43)

Ready for walk-forward validation and modeling! ✓


In [39]:
# Detailed missing values analysis and cleaning
print("=" * 70)
print("MISSING VALUES ANALYSIS & CLEANING")
print("=" * 70)

missing_counts = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_counts.index,
    'Missing_Count': missing_counts.values,
    'Missing_Percentage': missing_pct.values
})

# Filter only columns with missing values
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print("\nColumns with missing values:")
    print(missing_df.to_string(index=False))
    print(f"\nTotal rows with at least one missing value: {df.isnull().any(axis=1).sum()}")
    print(f"Rows before cleaning: {len(df)}")
else:
    print("\n✓ No missing values detected!")

# Remove all missing values
df_clean = df.dropna()

print(f"\nRows after dropping NaN: {len(df_clean)}")
print(f"Rows removed: {len(df) - len(df_clean)}")
print(f"Data retention: {(len(df_clean)/len(df))*100:.2f}%")

print(f"\nFinal dataset shape: {df_clean.shape}")
print(f"Features: {df_clean.shape[1] - 1} (excluding target)")
print(f"Target: 1 (next-day gold return)")

# Verify no missing values
print(f"\nMissing values check: {df_clean.isnull().sum().sum()}")
print("✓ Clean data ready for modeling!")

# Save cleaned features
df_clean.to_csv("/Users/erion/Desktop/MSCF2/ADA/Capstone-Project/data/processed/modeling_dataset.csv")
print("\n✓ Saved to: data/processed/modeling_dataset.csv")

print("\n" + "=" * 70)
print("Ready for modeling!")
print("=" * 70)


MISSING VALUES ANALYSIS & CLEANING

Columns with missing values:
           Column  Missing_Count  Missing_Percentage
            ma_30             29            0.527369
           vol_30             29            0.527369
      momentum_30             29            0.527369
           vol_20             19            0.345517
            ma_20             19            0.345517
      momentum_20             19            0.345517
           rsi_14             14            0.254592
           vol_10              9            0.163666
            ma_10              9            0.163666
      momentum_10              9            0.163666
        gold_lag5              5            0.090926
        gold_lag4              4            0.072740
         oil_lag3              3            0.054555
         spy_lag3              3            0.054555
         dxy_lag3              3            0.054555
      eurusd_lag3              3            0.054555
treasury_10y_lag3              3  